# Battle of the Neighborhood

This notebook is for the Applied Data Science Capstone course in Coursera. It's the last of the 9 courses provided by IBM as part of their Data Science Professional Certificate.

In [3]:
print('Hello Capstone Project!')

Hello Capstone Project!


In [5]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes #uncomment if geopy has not been installed 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes #uncomment if folium has not been installed 
import folium # map rendering library

#!pip install BeautifulSoup4
from urllib.request import urlopen
from bs4 import BeautifulSoup
import ssl
import csv

print('Libraries imported.')

Libraries imported.


In [7]:
#send a GET request to the wikipedia page consisting of list of Canada's postal codes 
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
source = requests.get(url)

#create soup object
soup = BeautifulSoup(source.text, 'lxml')

In [22]:
#scrape the table of Canada's postal codes using soup object
df = []
column_names = []
table = soup.find(class_='wikitable')

#index = index number of each row, tr = table row
for index, tr in enumerate(table.find_all('tr')):
    columns = []
    for value in tr.find_all(['th','td']):
        columns.append(value.text.rstrip())
    
    if (index == 0):
        column_names = columns #first row index is the header
    else:
        df.append(columns)
        
#convert list into Pandas DataFrame
canada_data = pd.DataFrame(data = df, columns = column_names)
canada_data.head()


,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [65]:
#get rid of all rows where borough is not assigned

canada_data = canada_data[canada_data['Borough'] != 'Not assigned']

#sort the table by postal code (ascending), for later use to append latitude and longitude
canada_data = canada_data.sort_values('Postal Code')

### The updated wikipedia table aggregates all the neighborhood of a unique postal code in a single row. Therefore, there's no need to aggregate any further.

In [66]:
#there's no need to aggregate further because number of rows in the dataframe
#is same as the number of unique postal codes 

print(canada_data.shape[0])
print(len(canada_data['Postal Code'].unique()))

103
103


### Moreover, after removing all the rows where the borough value is 'not assigned', there's no 'neighbourhood' value that is not assigned.

In [67]:
canada_data[canada_data['Neighbourhood'] == 'Not assigned']

,Postal Code,Borough,Neighbourhood


In [68]:
canada_data.shape

(103, 3)

In [71]:
import io #to extract the csv file of latitude and longitude of Canada's various postal codes

url_ll = 'https://cocl.us/Geospatial_data'
can_latlong = requests.get(url_ll).text
can_latlong = pd.read_csv(io.StringIO(can_latlong))
can_latlong.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [75]:
can_latlong['Postal Code'].nunique()

103

In [81]:
ca = pd.merge(canada_data, can_latlong, how='inner', on = 'Postal Code')

In [82]:
ca.shape

(103, 5)

In [83]:
ca.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [84]:
address = 'Canada'

geolocator = Nominatim(user_agent="canada_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Downtown Toronto are 61.0666922, -107.9917071.


In [92]:
# create map of Canada using latitude and longitude values
map_canada = folium.Map(location = [latitude, longitude], zoom_start = 0)

# add markers to map
for lat, lng, borough, neighborhood in zip(ca['Latitude'], ca['Longitude'], ca['Borough'], ca['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_canada)  
    
map_canada